<a href="https://colab.research.google.com/github/Nicordaro/ml4iot_hw3/blob/main/big_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install tensorflow_model_optimization

In [14]:
import os
# Clone github repository with data
# if os.path.isdir('./Project_MLDL'):
!rm -rf Project_MLDL
DATA_DIR = './ml4iot_hw3'
if not os.path.isdir(DATA_DIR):
  !git clone https://github.com/Nicordaro/ml4iot_hw3


In [15]:
import argparse
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import zlib
import shutil

'''
parser = argparse.ArgumentParser()
parser.add_argument('--version', type=str, required=False,
        help='model version: a,b or c')
args = parser.parse_args()

version = args.version
'''

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

zip_path = tf.keras.utils.get_file(
                origin = 'http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip',
                fname = 'mini_speech_commands.zip',
                extract = True,
                cache_dir = '.',
                cache_subdir = 'data')

data_dir = os.path.join('.','data', 'mini_speech_commands')

def filenames(path) :
    with open(path, "rb") as fp:
        arr = np.array(fp.read().splitlines())
    return arr
    
train_files = filenames(DATA_DIR+"/kws_train_split.txt")
val_files = filenames(DATA_DIR+"/kws_val_split.txt")
test_files = filenames(DATA_DIR+"/kws_test_split.txt")

LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

In [16]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step, num_mel_bins=None, lower_frequency=None,
                upper_frequency=None, num_coefficients=None, mfcc=False):
        
        self.labels = labels
        self.sampling_rate = sampling_rate 
        self.frame_length = frame_length  
        self.frame_step = frame_step       
        self.num_mel_bins = num_mel_bins   
        self.lower_frequency = lower_frequency  
        self.upper_frequency = upper_frequency  
        self.num_coefficients = num_coefficients  
        self.mfcc = mfcc  
        
        if mfcc is True:
            self.preprocess = self.preprocess_with_mfcc
        else :
            self.preprocess = self.preprocess_with_stft
            
            
    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]  
        label_id = tf.argmax(label == self.labels)
        
        audio_binary = tf.io.read_file(file_path)
        audio,_ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)  
        audio = self.pad(audio)
        
        return audio, label_id
    
    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])
        
        return audio
      
    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        
        stft = tf.signal.stft(audio, frame_length=self.frame_length, frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)
        
        if self.mfcc == False:
            spectrogram = tf.expand_dims(spectrogram, -1)
            spectrogram = tf.image.resize(spectrogram, [50,50])
            
        return spectrogram, label
    
    
    def preprocess_with_mfcc(self, file_path):
        
        spectrogram, label = self.preprocess_with_stft(file_path)
        
        num_spectrogram_bins = spectrogram.shape[-1]
        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(self.num_mel_bins, num_spectrogram_bins,
                                                                            self.sampling_rate, 
                                                                            self.lower_frequency, self.upper_frequency)
        
        mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)[..., :self.num_coefficients]
        mfccs = tf.expand_dims(mfccs, -1)
          
        return mfccs, label
        
    
    def make_ds(self, filenames, train):
        ds = tf.data.Dataset.from_tensor_slices(filenames)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        
        if train == True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)
            
        return ds

In [348]:
# Big model parameters
sampling_rate = 16000
frame_length = 640
frame_step = 320
num_mel_bins = 40
lower_frequency = 20
upper_frequency = 4000
num_mfccs = 12

In [349]:
# MFCC
generator = SignalGenerator(LABELS, sampling_rate, frame_length, frame_step, num_mel_bins, 
                                lower_frequency, upper_frequency, num_mfccs, mfcc=True)                  

train_ds = generator.make_ds(train_files, True)
val_ds = generator.make_ds(val_files, False)
test_ds = generator.make_ds(test_files, False)

In [140]:
#temp = tf.squeeze(next(iter(train_ds))[0][0]).shape

In [141]:
#shape1 = temp[0]
#shape2 = temp[1]  # = num_mfccs
#shape1, shape2

(49, 20)

In [379]:
def model_creation(name, alpha, mfcc=True):
    
    stride = [1,1]
    input_shape = (shape1,shape2) 
        
    if name == "cnn":
        model = keras.Sequential([
            keras.layers.Conv2D(filters = int(128*alpha), kernel_size=[3,3], strides=stride, use_bias=False, activation='relu'), 
            keras.layers.BatchNormalization(momentum=0.1),
            keras.layers.Conv2D(filters = int(128*alpha), kernel_size=[3,3], strides=[1,1], use_bias=False, activation='relu'), 
            keras.layers.BatchNormalization(momentum=0.1),
            keras.layers.Conv2D(filters = int(128*alpha), kernel_size=[3,3], strides=[1,1], use_bias=False, activation='relu'), 
            keras.layers.BatchNormalization(momentum=0.1),
            keras.layers.Conv2D(filters = int(128*alpha), kernel_size=[3,3], strides=[1,1], use_bias=False, activation='relu'), 
            keras.layers.BatchNormalization(momentum=0.1),
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(8)
        ])
    
    return model

In [380]:
alpha = 1
mfcc = True
#number_of_clusters = 8
    
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.02)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()
n_epochs = 20
model_name = 'cnn'

def scheduler(epoch, lr):
    if epoch == 10 or epoch == 14:
        return lr*0.1   
    return lr

In [381]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', min_delta = 0.005, patience = 5)

checkpoint = tf.keras.callbacks.ModelCheckpoint( './solutions Group16/Group16_big', 
                                                 save_best_only = True,
                                                 save_weights_only = True,
                                                 monitor = 'val_loss',
                                                 mode = 'min')

schedule_callback = tf.keras.callbacks.LearningRateScheduler(schedule = scheduler)

In [382]:
cnn = model_creation(model_name, alpha, mfcc = mfcc)

cnn.compile(optimizer = optimizer,
            loss = loss,              
            metrics = metric)

cnn.fit(train_ds, epochs = n_epochs, validation_data=val_ds, callbacks = [schedule_callback, checkpoint])

Epoch 1/20


ValueError: ignored

In [378]:
cnn.load_weights('./solutions Group16/Group16_big')
test_loss, test_accuracy = cnn.evaluate(test_ds, verbose=2)
test_accuracy

25/25 - 0s - loss: 0.1869 - sparse_categorical_accuracy: 0.9425


0.9424999952316284

In [369]:
# Save the model
cnn.save('./solutions Group16/big_model')

INFO:tensorflow:Assets written to: ./solutions Group16/big_model/assets


In [370]:
# Save the tflite
converter = tf.lite.TFLiteConverter.from_saved_model('./solutions Group16/big_model')
    
tflite_model = converter.convert()

with open('big_model.tflite', 'wb') as fp:
    fp.write(tflite_model)

In [ ]:
# Delete the directory with keras models and checkpoints
#shutil.rmtree('solutions Group16', ignore_errors=True)

In [371]:
# Test the model
test_ds_tflite = test_ds.unbatch().batch(1)

def acc_inference(interpreter):
    
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details() 
    output_details = interpreter.get_output_details()
    
    acc = tf.keras.metrics.SparseCategoricalAccuracy()
    
    for (x,y) in test_ds_tflite:
        interpreter.set_tensor(input_details[0]["index"], x)
        interpreter.invoke()
        y_pred = interpreter.get_tensor(output_details[0]["index"])
        
        y_true = y
        acc.update_state(y_true, y_pred)
    
    return acc.result().numpy()

In [373]:
interpreter = tf.lite.Interpreter(model_path = 'big_model.tflite')
tflite_acc = acc_inference(interpreter)
tflite_acc, os.path.getsize('big_model.tflite')

(0.9425, 1786852)